In [1]:
import psycopg2

# Connect to your PostgreSQL database
conn = psycopg2.connect(
    dbname="telecom", 
    user="postgres", 
    password="123456", 
    host="localhost", 
    port="5432"
)

In [2]:
import sys
import os

# Set the path to the parent directory (use raw string to handle backslashes)
sys.path.append(os.path.abspath(r'C:\Gedebie\finTech AI\week2\telecom_data_analysis'))

# Import the fetch_data function from the utils module
from utils import fetch_data

# Example usage
df = fetch_data(conn)

In [3]:
import pandas as pd
import numpy as np

# Assuming 'df' is the DataFrame with all the required columns

# Step 1: Handling missing values
# Fill numerical columns with the mean
df['TCP DL Retrans. Vol (Bytes)'] = df['TCP DL Retrans. Vol (Bytes)'].fillna(df['TCP DL Retrans. Vol (Bytes)'].mean())
df['TCP UL Retrans. Vol (Bytes)'] = df['TCP UL Retrans. Vol (Bytes)'].fillna(df['TCP UL Retrans. Vol (Bytes)'].mean())
df['Avg RTT DL (ms)'] = df['Avg RTT DL (ms)'].fillna(df['Avg RTT DL (ms)'].mean())
df['Avg RTT UL (ms)'] = df['Avg RTT UL (ms)'].fillna(df['Avg RTT UL (ms)'].mean())
df['Avg Bearer TP DL (kbps)'] = df['Avg Bearer TP DL (kbps)'].fillna(df['Avg Bearer TP DL (kbps)'].mean())
df['Avg Bearer TP UL (kbps)'] = df['Avg Bearer TP UL (kbps)'].fillna(df['Avg Bearer TP UL (kbps)'].mean())

# Fill categorical column "Handset Type" with the mode
df['Handset Type'] = df['Handset Type'].fillna(df['Handset Type'].mode()[0])

# Step 2: Define aggregation functions
aggregation_functions = {
    'TCP DL Retrans. Vol (Bytes)': 'mean',
    'TCP UL Retrans. Vol (Bytes)': 'mean',
    'Avg RTT DL (ms)': 'mean',
    'Avg RTT UL (ms)': 'mean',
    'Avg Bearer TP DL (kbps)': 'mean',
    'Avg Bearer TP UL (kbps)': 'mean',
    'Handset Type': lambda x: x.mode()[0]  # Mode for categorical column
}

# Step 3: Group by 'MSISDN/Number' (per customer) and aggregate data
df_aggregated = df.groupby('MSISDN/Number').agg(aggregation_functions).reset_index()

# Step 4: Create derived metrics
# Calculate Average TCP Retransmission (DL + UL)
df_aggregated['Avg TCP Retransmission (Bytes)'] = (df_aggregated['TCP DL Retrans. Vol (Bytes)'] + df_aggregated['TCP UL Retrans. Vol (Bytes)']) / 2

# Calculate Average RTT (DL + UL)
df_aggregated['Avg RTT (ms)'] = (df_aggregated['Avg RTT DL (ms)'] + df_aggregated['Avg RTT UL (ms)']) / 2

# Calculate Average Throughput (DL + UL)
df_aggregated['Avg Throughput (kbps)'] = (df_aggregated['Avg Bearer TP DL (kbps)'] + df_aggregated['Avg Bearer TP UL (kbps)']) / 2

# Step 5: Drop unnecessary columns after aggregating
df_aggregated = df_aggregated[['MSISDN/Number', 'Handset Type', 'Avg TCP Retransmission (Bytes)', 'Avg RTT (ms)', 'Avg Throughput (kbps)']]

# Output the aggregated DataFrame
print(df_aggregated)


        MSISDN/Number                      Handset Type  \
0        3.360100e+10    Huawei P20 Lite Huawei Nova 3E   
1        3.360100e+10            Apple iPhone 7 (A1778)   
2        3.360100e+10                         undefined   
3        3.360101e+10           Apple iPhone 5S (A1457)   
4        3.360101e+10           Apple iPhone Se (A1723)   
...               ...                               ...   
106851   3.379000e+10               Huawei Honor 9 Lite   
106852   3.379000e+10       Apple iPhone 8 Plus (A1897)   
106853   3.197021e+12  Quectel Wireless. Quectel Ec25-E   
106854   3.370000e+14                  Huawei B525S-23A   
106855   8.823971e+14  Quectel Wireless. Quectel Ec21-E   

        Avg TCP Retransmission (Bytes)  Avg RTT (ms)  Avg Throughput (kbps)  
0                         1.078479e+07     23.000000                   38.0  
1                         1.078479e+07     15.500000                   49.5  
2                         1.078479e+07     63.729294     

In [4]:
import pandas as pd

# Assuming 'df' is the DataFrame containing the required columns

# Step 1: Derive the necessary columns
df['TCP Retransmission (Bytes)'] = (df['TCP DL Retrans. Vol (Bytes)'] + df['TCP UL Retrans. Vol (Bytes)']) / 2
df['RTT (ms)'] = (df['Avg RTT DL (ms)'] + df['Avg RTT UL (ms)']) / 2
df['Throughput (kbps)'] = (df['Avg Bearer TP DL (kbps)'] + df['Avg Bearer TP UL (kbps)']) / 2

# Function to compute and list top, bottom, and most frequent values
def compute_top_bottom_frequent(df, column_name, n=10):
    top_n = df.nlargest(n, column_name)[column_name].tolist()  # Top n values
    bottom_n = df.nsmallest(n, column_name)[column_name].tolist()  # Bottom n values
    most_frequent = df[column_name].value_counts().head(n).index.tolist()  # Most frequent n values
    
    return top_n, bottom_n, most_frequent

# Step 2: Compute top, bottom, and most frequent for each metric
tcp_top, tcp_bottom, tcp_frequent = compute_top_bottom_frequent(df, 'TCP Retransmission (Bytes)')
rtt_top, rtt_bottom, rtt_frequent = compute_top_bottom_frequent(df, 'RTT (ms)')
throughput_top, throughput_bottom, throughput_frequent = compute_top_bottom_frequent(df, 'Throughput (kbps)')

# Step 3: Display the results
print("TCP Retransmission (Bytes):")
print("Top 10:", tcp_top)
print("Bottom 10:", tcp_bottom)
print("Most Frequent 10:", tcp_frequent)

print("\nRTT (ms):")
print("Top 10:", rtt_top)
print("Bottom 10:", rtt_bottom)
print("Most Frequent 10:", rtt_frequent)

print("\nThroughput (kbps):")
print("Top 10:", throughput_top)
print("Bottom 10:", throughput_bottom)
print("Most Frequent 10:", throughput_frequent)


TCP Retransmission (Bytes):
Top 10: [2172057825.5, 2150738719.5, 2147215982.5, 2145732475.0, 2144060430.0, 2137745856.5, 2134323675.0, 2130808019.5, 2130087072.5, 2127329821.5]
Bottom 10: [43.0, 48.5, 53.0, 54.0, 56.5, 64.0, 64.5, 67.0, 67.0, 71.5]
Most Frequent 10: [10784786.467576498, 10405604.135170965, 380494.33240553306, 10405622.135170965, 379848.33240553306, 379875.33240553306, 10405616.135170965, 381159.33240553306, 380488.33240553306, 10406287.135170965]

RTT (ms):
Top 10: [48462.0, 32320.5, 27906.0, 27424.0, 13639.0, 13150.0, 12961.0, 12857.5, 12694.0, 12369.0]
Bottom 10: [0.0, 0.0, 0.0, 0.0, 1.0, 2.0, 2.0, 2.5, 3.0, 3.0]
Most Frequent 10: [63.729294482330296, 14.5, 19.5, 19.0, 20.0, 15.0, 14.0, 24.5, 20.5, 15.5]

Throughput (kbps):
Top 10: [191131.0, 156622.0, 152149.5, 150273.0, 141965.5, 140572.0, 138576.0, 138102.5, 137026.0, 134944.0]
Bottom 10: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Most Frequent 10: [31.5, 7.5, 48.5, 45.0, 49.0, 48.0, 49.5, 44.5, 45.5, 46.5

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Select the relevant metrics for clustering
X = df[['Avg Bearer TP DL (kbps)', 'TCP DL Retrans. Vol (Bytes)', 'Avg RTT DL (ms)']]

# Normalize the data to ensure all metrics are on the same scale
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Perform K-means clustering with k = 3
kmeans = KMeans(n_clusters=3, random_state=42)
df['Cluster'] = kmeans.fit_predict(X_scaled)

# Analyze the clustering results by viewing cluster assignments
print(df[['Avg Bearer TP DL (kbps)', 'TCP DL Retrans. Vol (Bytes)', 'Avg RTT DL (ms)', 'Cluster']].head())

# Summary statistics for each cluster
cluster_summary = df.groupby('Cluster').mean()[['Avg Bearer TP DL (kbps)', 'TCP DL Retrans. Vol (Bytes)', 'Avg RTT DL (ms)']]
print(cluster_summary)


In [8]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Example data (replace with your actual dataset)
data = {
    'Avg Bearer TP DL (kbps)': [23.0, 16.0, 6.0, 44.0, 6.0],
    'TCP DL Retrans. Vol (Bytes)': [20809910.0, 20809910.0, 20809910.0, 20809910.0, 20809910.0],
    'Avg RTT DL (ms)': [42.0, 65.0, 109.8, 109.8, 109.8]
}
df = pd.DataFrame(data)

# Number of clusters for k-means
k = 3

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[['Avg Bearer TP DL (kbps)', 'TCP DL Retrans. Vol (Bytes)', 'Avg RTT DL (ms)']])

# Perform k-means clustering
kmeans = KMeans(n_clusters=k, random_state=42)
kmeans.fit(X_scaled)

# Get the cluster centers (centroids)
centroids = kmeans.cluster_centers_

# Assign the clusters to the dataset
df['Cluster'] = kmeans.labels_

# Define which cluster is the less engaged and worst experience
# (assuming cluster 0 is less engaged, and cluster 1 is the worst experience)
less_engaged_cluster = centroids[0]  # Replace with actual less engaged cluster if known
worst_experience_cluster = centroids[1]  # Replace with actual worst experience cluster if known

# Function to calculate Euclidean distance
def euclidean_distance(point, centroid):
    return np.linalg.norm(point - centroid)

# Calculate engagement and experience scores
df['Engagement Score'] = [euclidean_distance(x, less_engaged_cluster) for x in X_scaled]
df['Experience Score'] = [euclidean_distance(x, worst_experience_cluster) for x in X_scaled]

# Output the dataset with the engagement and experience scores
print(df)


   Avg Bearer TP DL (kbps)  TCP DL Retrans. Vol (Bytes)  Avg RTT DL (ms)  \
0                     23.0                   20809910.0             42.0   
1                     16.0                   20809910.0             65.0   
2                      6.0                   20809910.0            109.8   
3                     44.0                   20809910.0            109.8   
4                      6.0                   20809910.0            109.8   

   Cluster  Engagement Score  Experience Score  
0        0          0.473854          2.807419  
1        0          0.473854          2.536611  
2        2          2.195012          2.703274  
3        1          2.633147          0.000000  
4        2          2.195012          2.703274  


In [9]:
import pandas as pd

# Create the DataFrame
data = {
    "Avg Bearer TP DL (kbps)": [23.0, 16.0, 6.0, 44.0, 6.0],
    "TCP DL Retrans. Vol (Bytes)": [20809910.0, 20809910.0, 20809910.0, 20809910.0, 20809910.0],
    "Avg RTT DL (ms)": [42.0, 65.0, 109.8, 109.8, 109.8],
    "Cluster": [0, 0, 2, 1, 2],
    "Engagement Score": [0.473854, 0.473854, 2.195012, 2.633147, 2.195012],
    "Experience Score": [2.807419, 2.536611, 2.703274, 0.000000, 2.703274]
}

df = pd.DataFrame(data)

# Calculate Satisfaction Score
df['Satisfaction Score'] = (df['Engagement Score'] + df['Experience Score']) / 2

# Sort by Satisfaction Score in descending order
df_sorted = df.sort_values(by='Satisfaction Score', ascending=False)

# Get the top 10 satisfied customers
top_10_satisfied_customers = df_sorted.head(10)

print(top_10_satisfied_customers)


   Avg Bearer TP DL (kbps)  TCP DL Retrans. Vol (Bytes)  Avg RTT DL (ms)  \
2                      6.0                   20809910.0            109.8   
4                      6.0                   20809910.0            109.8   
0                     23.0                   20809910.0             42.0   
1                     16.0                   20809910.0             65.0   
3                     44.0                   20809910.0            109.8   

   Cluster  Engagement Score  Experience Score  Satisfaction Score  
2        2          2.195012          2.703274            2.449143  
4        2          2.195012          2.703274            2.449143  
0        0          0.473854          2.807419            1.640636  
1        0          0.473854          2.536611            1.505232  
3        1          2.633147          0.000000            1.316574  


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Create the DataFrame
data = {
    "Avg Bearer TP DL (kbps)": [23.0, 16.0, 6.0, 44.0, 6.0],
    "TCP DL Retrans. Vol (Bytes)": [20809910.0, 20809910.0, 20809910.0, 20809910.0, 20809910.0],
    "Avg RTT DL (ms)": [42.0, 65.0, 109.8, 109.8, 109.8],
    "Cluster": [0, 0, 2, 1, 2],
    "Engagement Score": [0.473854, 0.473854, 2.195012, 2.633147, 2.195012],
    "Experience Score": [2.807419, 2.536611, 2.703274, 0.000000, 2.703274]
}

df = pd.DataFrame(data)

# Calculate Satisfaction Score
df['Satisfaction Score'] = (df['Engagement Score'] + df['Experience Score']) / 2

# Features and target variable
X = df[["Avg Bearer TP DL (kbps)", "TCP DL Retrans. Vol (Bytes)", "Avg RTT DL (ms)", "Cluster", "Engagement Score"]]
y = df["Satisfaction Score"]

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Display the coefficients
coefficients = pd.DataFrame(model.coef_, X.columns, columns=['Coefficient'])
print(coefficients)


Mean Squared Error: 0.19884546028615777
R-squared: nan
                             Coefficient
Avg Bearer TP DL (kbps)        -0.029779
TCP DL Retrans. Vol (Bytes)     0.000000
Avg RTT DL (ms)                 0.004438
Cluster                         0.000885
Engagement Score               -0.000218


C:\Users\CraftSoft\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:1211: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
